## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.4.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.8603333234786987

Best val_accuracy So Far: 0.906000018119812
Total elapsed time: 00h 01m 50s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                1179680   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 1,226,954
Trainable params: 1,226,954
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1747 - accuracy: 0.9374 - val_loss: 0.2729 - val_accuracy: 0.9025
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1314 - accuracy: 0.9520 - val_loss: 0.2673 - val_accuracy: 0.9110
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1031 - accuracy: 0.9629 - val_loss: 0.3029 - val_accuracy: 0.9123
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0759 - accuracy: 0.9726 - val_loss: 0.3449 - val_accuracy: 0.9042
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0609 - accuracy: 0.9771 - val_loss: 0.3558 - val_accuracy: 0.9077
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0454 - accuracy: 0.9839 - val_loss: 0.4009 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0401 - accuracy: 0.9860 - val_loss: 0.4662 - val_accuracy